# Transformers for Text Summerization

As we discussed earlier transformers are great for many NLP tasks and text summerization is one such task.

In 2019, a team of researchers formed a new model based on the assertion every NLP problem can be represented as a Text-To-Text function(and if we in fact think about it, most of the tasks we expect a transformer to do falls in to such category). The idea was to train a transformer using transfer learning techniques during the training phase and finetune using text to text approach. 

This idea led to new performance level for transformer models and the model is called Text-To-Text Transfer Transformer or T5 for short. The researchers wanted the T5 model to be have task agnostic training process. To acheive that purpose they simply added a prefix to the sequences defining what need to be done. For example,

- 'translate English to German: \[sequence]'
- 'summerize: \[sequence]'

This way T5 models can get different tasks in single format. This led T5 to be used in many use cases with same parameters.
